In [2]:
import os
import dotenv
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper

dotenv_file = dotenv.find_dotenv("/home/sangbin_yun/dev/llm_agent/.env")
dotenv.load_dotenv(dotenv_file)

True

### SerpAPI

In [ ]:
# serp api
tool = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper())
test = tool.run('EEG AND "conflict" AND "PLV"')

### Scholary

In [ ]:
from scholarly import scholarly, ProxyGenerator

# Set up a ProxyGenerator object to use free proxies
# This needs to be done only once per session
# pg = ProxyGenerator()
# pg.FreeProxies()
# scholarly.use_proxy(pg)

search_query = scholarly.search_pubs('EEG AND "conflict" AND "PLV"')
print(search_query.total_results)

In [ ]:
total_eprint = 0
not_total_eprint = 0
for i in range(200):
    test = next(search_query)
    # print(f"===================={i}====================")
    # print(test["bib"]["abstract"])
    # print(test["pub_url"])
    try:
        # print(test["eprint_url"])
        total_eprint += 1
    except:
        # print("No eprint_url")
        not_total_eprint += 1

### Custom Scraping

In [73]:
# User Agent settnig
# https://useragentstring.com/pages/Chrome/

# whatismybrowser API as a user agent
# https://developers.whatismybrowser.com/api/docs/v2/integration-guide/#software-version-numbers
headers = {
    "X-API-KEY": os.getenv("X-API-KEY"),  # Your API Key goes here.
}

# Fake user agent setting
from fake_useragent import UserAgent
ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (iPhone; CPU iPhone OS 17_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) EdgiOS/120.0.2210.86 Version/17.0 Mobile/15E148 Safari/604.1


In [10]:
# proxy
from fp.fp import FreeProxy
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
ua = UserAgent()

def get_http_proxy(https = False):
    while True:
        try:
            proxy = FreeProxy(country_id = ['KR'], rand = True, https = https).get()
            return proxy1
        except:
            None

proxy = get_http_proxy(False)
proxy2 = get_http_proxy(True)

headers = {
    "User-Agent": ua.random
}
proxies = {
    'http': proxy,
    'https': proxy2,
}
params = {
    'q': 'EEG ang plv',  # search query
    'hl': 'en'       # language of the search
}

html = requests.get(
    'https://scholar.google.com/scholar', 
    headers = headers,
    proxies = proxies, 
    params = params
).text
soup = BeautifulSoup(html, 'lxml')

In [12]:
from bs4 import BeautifulSoup
import requests, lxml, os, json

def get_http_proxy(https = False):
    return FreeProxy(country_id = ['KR'], rand = True, https = https).get()

def scrape_one_google_scholar_page():
    # headers = {
    #     'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
    # }
    proxies = {
        'http': get_http_proxy(False),
        'https': get_http_proxy(True),
    }
    params = {
        'q': 'samsung',  # search query
        'hl': 'en'       # language of the search
    }

    html = requests.get('https://scholar.google.com/scholar', proxies = proxies, params = params).text
    soup = BeautifulSoup(html, 'lxml')

    # JSON data will be collected here
    data = []

    # Container where all needed data is located
    for result in soup.select('.gs_r.gs_or.gs_scl'):
        title = result.select_one('.gs_rt').text
        title_link = result.select_one('.gs_rt a')['href']
        publication_info = result.select_one('.gs_a').text
        snippet = result.select_one('.gs_rs').text
        cited_by = result.select_one('#gs_res_ccl_mid .gs_nph+ a')['href']
        try:
            pdf_link = result.select_one('.gs_or_ggsm a:nth-child(1)')['href']
        except: 
            pdf_link = None

        data.append({
            'title': title,
            'title_link': title_link,
            'publication_info': publication_info,
            'snippet': snippet,
            'cited_by': f'https://scholar.google.com{cited_by}',
            'related_articles': f'https://scholar.google.com{related_articles}',
            'all_article_versions': f'https://scholar.google.com{all_article_versions}',
            "pdf_link": pdf_link
        })

    print(json.dumps(data, indent = 2, ensure_ascii = False))

scrape_one_google_scholar_page()

KeyboardInterrupt: 

### Beautiful Soup

In [28]:
import requests
from bs4 import BeautifulSoup
from langchain_core.documents import Document




b"{'author_id': ['2F522iUAAAAJ', '', 'hqwQyRsAAAAJ', ''],\n 'bib': {'abstract': 'measured the effect of a conflict in combination with '\n                     'extended  Male rats were subjected to  a 1-h social '\n                     'conflict followed by 5  consisting of a social conflict '\n                     'followed by gentle handling, as',\n         'author': ['P Meerlo', 'EA de Bruin', 'AM Strijkstra', 'S Daan'],\n         'pub_year': '2001',\n         'title': 'A social conflict increases EEG slow-wave activity during '\n                  'subsequent sleep',\n         'venue': 'Physiology & behavior'},\n 'citedby_url': '/scholar?cites=336417141359410019&as_sdt=5,33&sciodt=0,33&hl=en',\n 'eprint_url': 'https://www.researchgate.net/profile/Peter-Meerlo/publication/11903429_A_social_conflict_increases_EEG_slow-wave_activity_during_subsequent_sleep/links/59f443a4a6fdcc075ec36647/A-social-conflict-increases-EEG-slow-wave-activity-during-subsequent-sleep.pdf',\n 'filled': False,\

### Document loader test

In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

DATA_PATH = "/mnt/c/Users/beene/Downloads"

loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
print(f"Processed {len(documents)} pdf files")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts=text_splitter.split_documents(documents)

Processed 10 pdf files


In [11]:
dir(documents[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo